## Integração do modelo do Bedrock com os agentes do LangChain

Algumas aplicações exigem uma sequência adaptável de chamadas para modelos de linguagem e diversos recursos dependendo de entrada do usuário. A interface do agente possibilita ter a flexibilidade necessária para essas aplicações. Um agente tem disponibilidade para uma gama de recursos e seleciona quais utilizar com base nas entradas do usuário. Os agentes conseguem utilizar várias ferramentas e usar o resultado de uma ferramenta como entrada para a outra.  

Existem duas categorias principais de agentes:

- Agentes de ação: em cada intervalo, eles determinam a ação subsequente usando os resultados de todas as ações anteriores. 
- Agentes plan-and-execute: determinam a ordem completa das ações inicialmente, depois implementam todas sem atualizar o plano.

Neste caderno, demonstraremos o uso de agentes `plan-and-execute` juntamente com o `Zero-shot ReAct`, um agente de ação que usa o framework [`ReAct`](https://arxiv.org/pdf/2205.00445.pdf) para selecionar a ferramenta apropriada com base exclusivamente na descrição da ferramenta. Para isso, você precisa fornecer a descrição de cada ferramenta. 

## Arquitetura
![](./images/arch-agents.png)

In [ ]:
import json
import os
import sys
import boto3

# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
os.environ['SERPAPI_API_KEY'] = "<YOUR_SERP_API_KEY>" # Required for the search tool


In [ ]:
model_parameter = {"temperature": 0.0, "top_p": .5, "max_tokens_to_sample": 2000}

## Como usar o ReAct: sinergia de raciocínio e atuação no framework de modelos de linguagem
Modelos grandes de linguagem podem gerar explicações para seus raciocínios e respostas específicas para tarefas alternadamente. 

A produção de explicações de raciocínio permite que o modelo suponha, monitore e revise planos de ação e até mesmo lide com cenários inesperados. A etapa de ação permite que o modelo faça a interface e obtenha informações de fontes externas, como ambientes ou bases de conhecimento.

O framework ReAct pode permitir que modelos grandes de linguagem interajam com ferramentas externas para obter informações que resultem em respostas mais precisas e baseadas em fatos.

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.llms.bedrock import Bedrock
from langchain import LLMMathChain
from langchain.utilities import SerpAPIWrapper

Carregue dois objetos de LLM únicos com parâmetros de modelo únicos, um para o agente e outro para a cadeia matemática.

Um LLM único com sequências de interrupção para cadeias matemáticas ajuda a evitar que o Claude seja excessivamente detalhado ao executar a cadeia matemática.

Além disso, adaptaremos o modelo padrão para funcionar melhor com o Claude, já que os modelos padrão do LangChain não se encaixam tão bem por padrão, então incluiremos a ferramenta recém-criada à nossa lista de ferramentas.

In [ ]:
react_agent_llm = Bedrock(model_id="anthropic.claude-instant-v1", model_kwargs=model_parameter)
math_chain_llm = Bedrock(model_id="anthropic.claude-instant-v1",
                         model_kwargs={"temperature":0,"stop_sequences" : ["```output"]})

tools = load_tools(["serpapi"], llm=react_agent_llm)

llm_math_chain = LLMMathChain(llm=math_chain_llm, verbose=True)

llm_math_chain.llm_chain.prompt.template = """Human: Given a question with a math problem, provide only a single line mathematical expression that solves the problem in the following format. Don't solve the expression only create a parsable expression.
```text
${{single line mathematical expression that solves the problem}}
```

Assistant:
 Here is an example response with a single line mathematical expression for solving a math problem:
```text
37593**(1/5)
```

Human: {question}

Assistant:"""

tools.append(
    Tool.from_function(
        func=llm_math_chain.run,
        name="Calculator",
        description="Useful for when you need to answer questions about math.",
    )
)

In [ ]:
react_agent = initialize_agent(tools, 
                               react_agent_llm, 
                               agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                               verbose=True,
                            #    max_iteration=2,
                            #    return_intermediate_steps=True,
                            #    handle_parsing_errors=True,
                               )

In [ ]:
prompt_template = """Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do, Also try to follow steps mentioned above
Action: the action to take, should be one of ["Search", "Calculator"]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Question: {input}

Assistant:
{agent_scratchpad}"""

In [ ]:
react_agent.agent.llm_chain.prompt.template=prompt_template

In [ ]:
question = "What is Amazon SageMaker? What is the launch year multiplied by 2"

In [ ]:
react_agent(question)

## Geração do código da ferramenta
É possível usar GenAI para criar o código nas ferramentas? Claro! Com cuidado, pois há variações no código e 
pode não ser seguro para a execução sem análise humana.

O exemplo a seguir usa o código gerado pelo Claude para gerar a ferramenta. Substitui a ferramenta EC2Search acima pelo código boto python.

In [ ]:
import xml.etree.ElementTree as ET
from IPython.display import display, Markdown, Latex

boto3_bedrock = boto3.client(service_name='bedrock-runtime',
                             region_name=os.environ["AWS_DEFAULT_REGION"])

prompt_data = """
Human: You are an AI python code generator. You write really great code.

Write a python function named list_tagged_instances with one parameter named tagname. 

The function queries the boto3 library to return a list all of the EC2 instances that have a tag equal to the tagname parameter. 

return the code inside <code></code>.
Assistant:"""

body = json.dumps({"prompt": prompt_data, "max_tokens_to_sample": 500})
modelId = "anthropic.claude-instant-v1"  
accept = "application/json"
contentType = "application/json"

response = boto3_bedrock.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())

tree = ET.ElementTree(ET.fromstring(response_body.get("completion")))
python_code = tree.getroot().text

display(Markdown(f'```{python_code}```'))

exec(python_code)

tools = [
    Tool.from_function(
        func=list_tagged_instances,
        name="List",
        description="Queries the boto3 library to return a list all of the EC2 instances that have a tag equal to the tagname parameter."
    ),
]

## Execução do código
Se o código acima parecer razoável, podemos usá-lo para executar nosso agente.

**Observação: para este trabalho, você precisa de uma instância do EC2 nesta conta com uma tag chamada “delete” e algum valor na tag. Ela diferencia maiúsculas e minúsculas, então, antes de executar o trabalho, crie uma instância do EC2 e a defina para o estado interrompido.

In [ ]:
llm = Bedrock(model_id="anthropic.claude-instant-v1", client=boto3_bedrock, model_kwargs=model_parameter)

react_agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

question = """Human: Please list my EC2 instances with the tag delete.
Assistant:"""

result = react_agent.run(question)

print(f"{result}")

## Outra variação — Pesquisar por uma instância que não existe
O prompt muda para pesquisar por instâncias do EC2 com uma tag que não existe, então nenhuma instância deve aparecer.

In [ ]:
question = """Human: Please list my EC2 instances with the tag doesnotexist.
Assistant:"""

result = react_agent(question)

print(f"{result}")

## Ferramentas de banco de dados
Um uso comum para o agente é pesquisar um registro em um banco de dados. Não seria prático incluir todo o banco de dados no contexto, então você pode fornecer ferramentas que realizam ações com base no banco de dados e eliminam alucinações, ao mesmo tempo em que mantêm as interações conversacionais.

### Agente do banco de dados SQL
O LangChain tem um agente de banco de dados SQL para demonstração de como fazer perguntas a uma DB para obter respostas. Para saber mais detalhes, leia este documento: https://python.langchain.com/docs/integrations/toolkits/sql_database

O agente carregará o esquema do DB no contexto e gerará declarações do SQL com base em perguntas de linguagem natural. Em seguida, a declaração do SQL é executada com referência ao banco de dados e o resultado é enviado.

### Agentes de dados
Enquanto o agente de banco de dados SQL é útil para exploração de dados e geração de consultas, também há casos em que você gostaria de 
Para entidades específicas, pode ser criada uma ferramenta para recuperar dados do banco de dados e fornecer próximas etapas de contexto no prompt.

O exemplo a seguir simulará uma consulta de DB para um cliente na tabela de clientes. Substitua esse código por um DynamoDB ib de pesquisa ou um banco de dados de relação.

In [ ]:
customer_table=[
  {
    "id": 1, 
    "first_name": "John", 
    "last_name": "Doe",
    "age": 35,
    "postal_code": "90210"
  },
  {  
    "id": 2,
    "first_name": "Jane",
    "last_name": "Smith", 
    "age": 27,
    "postal_code": "12345"
  },
  {
    "id": 3, 
    "first_name": "Bob",
    "last_name": "Jones",
    "age": 42,
    "postal_code": "55555"
  },
  {
    "id": 4,
    "first_name": "Sara", 
    "last_name": "Miller",
    "age": 29, 
    "postal_code": "13579"
  },
  {
    "id": 5,
    "first_name": "Mark",
    "last_name": "Davis",
    "age": 31,
    "postal_code": "02468"
  },
  {
    "id": 6,
    "first_name": "Laura",
    "last_name": "Wilson",
    "age": 24,
    "postal_code": "98765" 
  },
  {
    "id": 7,
    "first_name": "Steve",
    "last_name": "Moore",
    "age": 36,
    "postal_code": "11223"
  },
  {
    "id": 8,
    "first_name": "Michelle",
    "last_name": "Chen",
    "age": 22,
    "orders": [
        {
            "order_id": 1,
            "description": "An order of 1 dozen pencils"
        },
        {
            "order_id": 2,
            "description": "An order of 2 markers"
        }
    ],
    "postal_code": "33215"
  },
  {
    "id": 9,
    "first_name": "David",
    "last_name": "Lee",
    "age": 29,
    "postal_code": "99567"
  },
  {
    "id": 10,
    "first_name": "Jessica",
    "last_name": "Brown",
    "age": 18, 
    "postal_code": "43210"
  }
]

def customer_lookup(id):
    print(f"search by customer {id}")
    for customer in customer_table:
        if customer["id"] == int(id):
            print(f"found customer {id} {customer}")
            return customer
        
    return None

tools.append(Tool.from_function(
        name="CustomerLookup",
        func=customer_lookup,  # Mock Function, replace with an api call
        description="Use this when you need to lookup a customer by id."
    ))

In [ ]:
react_agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

question = """\n\nHuman: write one sentence summary about the information you know about the customer with an id of 2.

\n\nAssistant: Here is the one sentence summary: """

result = react_agent.run(question)

print(f"{result}")